In [ ]:
#
# Script to extract data from SEIS data Portal and to deglitch them using 
# SEISGlitch algorithm from Scholz et al. 
#
# @author sainton@ipgp.fr
# @date: january 2022
# @version: - 1.0: initial version
#           - 1.1: partial automation to run over severa sols  

from lxml import etree

import os
import sys
from os import path
import shutil
from datetime import datetime
import copy
import warnings
import configparser
import csv

# pandas to manage large matrices
import pandas as pd

# Internal lib
# LMST <-> UTC

from obspy import UTCDateTime

from MarsConverter import MarsConverter

# Read channel properties files
from modules.core.aspic_channel import read_csv_panda

# To get data from SEIS Data Portal
from modules.msds.msds_utils import get_dataless_from_portal, get_data_from_portal


####################################
#
# PARAMETERS SETRINGS
#

### Please consider to change this paths with your own paths
###
# Don't modify this
propertiesFile = './configuration/channel.properties2.0.csv'
msdspwd = "/Users/greg/.aspic/msdspwd.ini"
# Output file to prepare data extraction.
filename = "/Users/greg/CODE/seis_toolbox/configuration/chan_list_2_scan_scholz_2022_01_13.xml"
postfile_nm = "/Users/greg/CODE/seis_toolbox/configuration/postfile_scholz_2022_01_13.txt"

#data_dir = "/Users/greg/Dropbox (IPGP)/MARS/Deglitched_data_by_Scholz"
#data_dir2 = "/Users/greg/Dropbox\ \(IPGP\)/MARS/Deglitched_data_by_Scholz"

data_dir = "/Users/greg/Dropbox (IPGP)/MARS/Deglitch_BackUp"
data_dir2 = "/Users/greg/Dropbox\ \(IPGP\)/MARS/Deglitch_BackUp"

# Data to load
extract_mseed_from_SDP = True
verbose = True
export_data = True

station2list = ["ELYSE"]
list_channels_Sci = ["VBBVel"]
list_sps  = ["100"]
list_gain = ["HG"]

delta_t = 1800           # time in seconds before and after the UTC time interval

sol_bounds = [1222, 1222]
sol_list = [s for s in range(sol_bounds[0], sol_bounds[1]+1)]

#sol_list = [331, 1048] # 

# read some values from the config

try: 
    msdspwd
except:
    print("Please check the aspic_config_file.txt to set 'msdspwd' with the proper path to msdspwd.ini")
    msds_passwd_file = None
else:
    msds_passwd_file = msdspwd

# Load the configuration file
with open(msds_passwd_file) as f:
    sample_config = f.read()

config = configparser.ConfigParser()
config.read(msds_passwd_file)
f.close()

# read some values from the config
MSDSlogin = config.get("default", "login")
MSDSpasswd = config.get("default", "password")

mDate = MarsConverter()

# --------
# Functions to be moved into a python file
# ---------

def create_dir_if_not_exists(path2dir):
    """ Create directory if it does not already exists
    
    -----
    INPUT:
        @path2dir: str - absoloute path of the directory to be created
    
    -----
    RETURN:
        None
    """
    if not os.path.isdir(path2dir):
        try:
            os.makedirs(path2dir)
        except OSerror as e:
            print("Directory already exists")
    else:
        print("This directory already exists")

def df_2_postfile(df, starttime, endtime, outputfile=None, verbose=False):
    """
    Convert a DataFrame to a postfile to extract
    data from MSDS InSight Database.
    
    ----
    INPUT:
        @df: Pandas dataframe containing informations about channels:
                    network, station, location, channel
                 eg:   XB     ELYSE      02       MHU
        @starttime: Datetime format start date for the request
        @endtime: Datetime format end date for the request

    -----
    OUPUT:
        None
    """
    
    if starttime < endtime:
        df.loc[:, "starttime"] = starttime
        df.loc[:, "endtime"] = endtime
    else:
        sys.exit("startdate > enddate")
    
    sub_df = df[["Network", "Station", "locid", "chan", "starttime", "endtime"]]
    print("In df_2_postfile")
    print(sub_df)
    if outputfile is not None:
        sub_df.to_csv(f"{outputfile}", header=None, 
                      index=None, sep=' ')
        if verbose: 
            print(f"Postfile create: {outputfile}")
    return sub_df


# --------
# Extraction from MSDS 
# ---------
for sol in sol_list:
    
    #starttime = mDate.get_lmst_to_utc(lmst_date=sol)
    #endtime = mDate.get_lmst_to_utc(lmst_date=sol+1)

    starttime = UTCDateTime(2022, 5, 4, 22, 30, 0, 0)
    endtime = UTCDateTime(2022, 5, 5, 16, 0, 0, 0)
    
    print(starttime, endtime)
    
    starttime-=delta_t
    endtime+=delta_t
    # Location of the output data
    soldir = f"Sol{sol:04d}_100"
    path2dir = os.path.join(data_dir, soldir)
    print(path2dir)
    # Create the directory to save the output data
    create_dir_if_not_exists(path2dir)

    if extract_mseed_from_SDP:

        # read the properties file and fill a panda structure 
        channel_properties_list_pd = read_csv_panda(propertiesFile, ';')

        # Select the data according to the parameters
        chanlist_sci = channel_properties_list_pd.loc[channel_properties_list_pd["sensor"].isin(list_channels_Sci)
                            & channel_properties_list_pd["Station"].isin(station2list)
                            & channel_properties_list_pd["sps"].isin(list_sps)
                            & channel_properties_list_pd["gaintype"].isin(list_gain)]
        
        print(chanlist_sci)
        # Convert Sols to UTC and add delta_t seconds around
        #mDate = MarsConverter()
        #starttime = mDate.get_lmst_to_utc(lmst_date=sol) 
        #endtime = mDate.get_lmst_to_utc(lmst_date=sol+1)

        #starttime-=delta_t
        #endtime+=delta_t
        if verbose:
            print(f""" Interval of date for extraction for the sol {sol}
                     - starttime: {starttime}
                     - endtime: {endtime} """)

        # Create the postfile to extract data from MSDS
        df = df_2_postfile(chanlist_sci, starttime, endtime, 
                           outputfile=postfile_nm, 
                           verbose=True)
        if verbose:
            display(df.head(10))

        # Extract data and dataless from MSDS
        output_mseed = "VBB_UVW_2_scholz_prg.mseed"    # Don't change this filename -> It's used in SEISGLITCH config file
        output_dataless = "dataless_file_2_scholz_prg.xml"
        wget_code_ms = get_data_from_portal(postfile_nm, MSDSlogin, MSDSpasswd, 
                                            data_dir, output_mseed)
        wget_code_dl = get_dataless_from_portal(MSDSlogin, MSDSpasswd, "XB", "ELYSE",
                                                raw_dir_path = data_dir,
                                                outputfile = output_dataless,
                                                starttime = starttime,
                                                endtime = endtime,
                                                sol=sol)

        # Keep a copy wit the sol number into the filename
        mseed_original_file = os.path.join(data_dir, output_mseed)
        mseed_renamed_file  = mseed_original_file.replace("_prg.", "_prg_sol_{:04d}.".format(sol))
        output_renamed_mseed = output_mseed.replace("_prg.", "_prg_sol_{:04d}.".format(sol))
        shutil.copy(mseed_original_file, 
                    mseed_renamed_file)
        shutil.move(mseed_renamed_file, 
                    os.path.join(path2dir, output_renamed_mseed))
        if verbose:
            print("Rename the mseed file to keep a copy")
            print(f"Original filename: {mseed_original_file}")
            print(f"Renamed filename: {output_renamed_mseed}")
            print(f"File moved to {path2dir}")

    # Detect the glitches
    filename_yml = "config_TestR2.yml"
    if verbose:
        print(f"""All the filename and parameters useful 
        to run SEISGLITCH are defined in the file: {filename_yml}""")

    #########
    #
    # GLITCHES DETECTION
    #
    ##########
    if verbose:
        print("Execute SEISGLITCH code in detection mode: ")

    action = "detect" 
    config_yml = os.path.join(data_dir2, "config_TestR2.yml")

    os.system("seisglitch {} {}".format(action, config_yml))

    # Rename the glitches file (but keep the original file)
    glitch_original_file = os.path.join(data_dir, "glitches_VBB_UVW_2_Scholz_prg.txt")
    glitch_renamed_fname = "glitches_VBB_UVW_2_Scholz_prg.txt".replace("_deglitched", "_deglitched_sol_{:04d}".format(sol))
    glitch_renamed_file  = glitch_original_file.replace("_prg.", "_prg_sol_{:04d}.".format(sol))
    shutil.copy(glitch_original_file, 
                glitch_renamed_file)
    shutil.move(glitch_renamed_file, 
                    os.path.join(path2dir, glitch_renamed_fname))

    #########
    #
    # GLITCHES REMOVAL
    #
    ##########

    # Remove the glitches 
    if verbose:
        print("Execute SEISGLITCH code in removal mode: ")
    action = "remove" 
    config_yml = os.path.join(data_dir2, filename_yml)

    os.system("seisglitch {} {}".format(action, config_yml))

    deglitch_original_file = os.path.join(data_dir, "VBB_UVW_2_scholz_prg_deglitched.mseed")
    deglitch_renamed_file  = deglitch_original_file.replace("_deglitched", "_deglitched_sol_{:04d}".format(sol))
    deglitch_renamed_fname = "VBB_UVW_2_scholz_prg_deglitched.mseed".replace("_deglitched", "_deglitched_sol_{:04d}".format(sol))
    shutil.copy(deglitch_original_file, 
                deglitch_renamed_file)
    shutil.move(deglitch_renamed_file, 
                    os.path.join(path2dir, deglitch_renamed_fname))
    